In [14]:
# %matplotlib inline
# from d2l import torch as d2l
import torch
import random
import tensorflow as tf
import random

# Scratch

### Generating Dataset

In [36]:
# pytorch
def synthetic_data_pytorch(w, b, num_examples):  #@save
    """Generate y = Xw + b + noise."""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data_pytorch(true_w, true_b, 1000)

In [32]:
print('features:', features[0],'\nlabel:', labels[0])


features: tensor([0.4505, 1.3035]) 
label: tensor([0.6615])


### Reading the Dataset

In [21]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

### Initializing Model Parameters 

initialize weights by sampling random numbers from a normal distribution with mean 0 and a standard deviation of 0.01, and setting the bias to 0.

In [22]:
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

### Loss Function

In [24]:
def squared_loss(y_hat, y):  #@save
    """Squared loss."""
    return (y_hat - y.reshape(y_hat.shape))**2 / 2

### Optimization

In [26]:
def sgd(params, lr, batch_size):  #@save
    """Minibatch stochastic gradient descent."""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

### Model

In [25]:
def linreg(X, w, b):  #@save
    """The linear regression model."""
    return torch.matmul(X, w) + b

In summary, we will execute the following loop:

    Initialize parameters  (w,b) 

    Repeat until done

        Compute gradient  g←∂(w,b)1|B|∑i∈Bl(x(i),y(i),w,b) 

        Update parameters  (w,b)←(w,b)−ηg

In [28]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss
batch_size = 10

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # Minibatch loss in `X` and `y`
        # Compute gradient on `l` with respect to [`w`, `b`]
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # Update parameters using their gradient
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.040928
epoch 2, loss 0.000159
epoch 3, loss 0.000048


# Concise

In [38]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data_pytorch(true_w, true_b, 1000)

In [39]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [40]:
next(iter(data_iter))

[tensor([[-0.9556, -0.9292],
         [ 1.4730, -1.1891],
         [ 0.1878,  2.1987],
         [ 2.3053, -0.0889],
         [-0.0446, -0.1616],
         [-0.7145, -0.6807],
         [-0.0232, -0.0854],
         [-1.0172,  0.4339],
         [ 0.7452, -0.0080],
         [-1.0203, -0.8865]]),
 tensor([[ 5.4459],
         [11.1873],
         [-2.8908],
         [ 9.1155],
         [ 4.6608],
         [ 5.0887],
         [ 4.4553],
         [ 0.7026],
         [ 5.7166],
         [ 5.1788]])]

### Model

In [41]:
from torch import nn
net = nn.Sequential(nn.Linear(2, 1))

###Params

In [42]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

### Loss

In [43]:
loss = nn.MSELoss()